
---

# 🏰 Django + Pydantic Schemas

### 🎯 Intent

Use **Pydantic v2** with **Django/DRF** to validate inputs and serialize ORM objects into safe, clean API DTOs.

---

### 🧩 Core Components

1. **🏗 Out Schemas from ORM**

   ```python
   class UserOut(BaseModel):
       model_config = ConfigDict(from_attributes=True)
       id: int
       email: str
       name: str
   # usage → UserOut.model_validate(django_user)
   ```

2. **📥 Input / Update DTOs**

   ```python
   class UserIn(BaseModel): email: str; name: str
   class UserUpdate(BaseModel): email: str|None=None; name: str|None=None
   ```

3. **🧱 Django View Example**

   ```python
   def create_user(request):
       try:
           dto = UserIn.model_validate_json(request.body)
       except ValidationError as e:
           return JsonResponse({"errors": e.errors()}, status=422)
       # save...
       return JsonResponse(UserOut.model_validate(user).model_dump())
   ```

4. **🧰 DRF APIView Example**

   ```python
   class UserView(APIView):
       def post(self, request):
           try: dto = UserIn.model_validate(request.data)
           except ValidationError as e:
               return Response({"errors": e.errors()}, status=422)
           return Response(UserOut.model_validate(user).model_dump(), status=201)
   ```

5. **🔒 Strictness & Extra Fields**

   * `extra="forbid"` to block unknown fields.
   * Use strict types where needed (IDs, money).

6. **📤 Aliases**

   * `Field(alias="external_id")` + `model_dump(by_alias=True)` for clean API names.

7. **🔗 Relations**

   * Use `select_related/prefetch_related` before validating → avoids lazy DB hits.

8. **🛡️ Secrets**

   * Mask with `@field_serializer`.
   * Don’t include passwords/tokens in Out schemas.

9. **🧪 Testing**

   * Assert `422` on bad input.
   * Snapshot `model_json_schema()` for stable contracts.

10. **⚙️ When to Use DRF Serializers**

* Still useful if you need browsable API, forms, or DRF’s relation handling.
* Combine both: Pydantic for DTOs, DRF for routing/auth.

---
